# Costweet : CNN + Fasttext

Ce notebook classe des phrases en 3 catégories "cher", "moyen","éco".

Il fait partie d'une série de notebook permettant de comparer différentes approches de classification en faisant varier les architectures de réseaux de neurones et les méthodes d'encodification des textes.

Il s'appuie sur un ensemble de données de petite taille.

Cet ensemble est vectorisé en utilisant la librairie fasttext de facebook (camdial.org).

Le réseau de neurones est architecturé de trois couches 2 de type convolution. Ce réseau est identique à celui utilisé par le notebook intitulé "Costweet : CNN + manuel encoding"

L'accuracy obtenu le 7 février 2018  est de 92,7% à comparer aux 85% obtenus avec avec un réseau sans convolution.

Keras uses tensorflow uses sklearn uses numpy and pandas

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Convolution1D, Flatten, Dropout, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
#for fasttext
from __future__ import print_function
from gensim.models import KeyedVectors





Using TensorFlow backend.
d:\code\ml\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


 DATA PROCESS
 les données ont été saisies sous excel et sauvegardées en csv utf-8 bizarement avec l'option séparateur virgules de microsoft
 


In [2]:
def getDataset():
	dataset = pd.read_csv("RuleCout2.csv", delimiter=";", encoding='utf-8')
	#print ( 'dataset loaded with shape',dataset.shape)
	X = dataset['text']
	Y = dataset['class']
	dataset.set_index('text') #pour avoir les classes dans un ordre quelconque
	return dataset, X, Y

# utile pour les colonnes sur une ligne et afficher toutes les lignes
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', 1000)

# on ajoutera peu à peu de nouvelles colonnes au dataset 
dataset, X, Y = getDataset()
print(dataset)

                         text  class
0             à prix modique     éco
1     aucun problème d'argent   cher
2                 avantageux     éco
3               bas de gamme     éco
4           bistrot pas cher     éco
5                         bon   cher
6                 bon marché     éco
7   bon rapport qualité prix   moyen
8                       cher    cher
9                         éco    éco
10                économique     éco
11             entre 10 et 20    éco
12             gastronomique    cher
13             haut de gamme    cher
14           j’ai les moyens    cher
15             je m’en fiche    cher
16              je m’en fous    cher
17             je suis riche    cher
18          jusqu'à cinquante   cher
19  le meilleur des meilleurs   cher
20                      luxe    cher
21                   luxueux    cher
22      ma bourse est pleine    cher
23     menu pas cher le midi     éco
24                   modique     éco
25       moins de cent euros    cher
2

# Chargement de fasttext
Fasttext needs 10 minutes to be loaded
C'est le temps de chargement du dataset fasttextwiki.fr.vec donne un vocabulaire de 1 115 449 mots

We are publishing pre-trained word vectors for 294 languages, trained on Wikipedia using fastText.
These vectors in dimension 300 were obtained using the skip-gram model described in Bojanowski et al. (2016) with default parameters. Le .bin pèse 5 gigas-octets !

origine #https://blog.manash.me/how-to-use-pre-trained-word-vectors-from-facebooks-fasttext-a71e6d55f27





In [3]:
# Creating the model
fr_model = KeyedVectors.load_word2vec_format('../data/fasttextwiki.fr.vec')

# Getting the tokens 
words = []
for word in fr_model.vocab:
    words.append(word)

# Printing out number of tokens available
print("Number of Tokens: {}".format(len(words)))





Number of Tokens: 1152449


In [4]:
# encode retourne un vecteur en fonction d'un mot
def score(word):
    return  fr_model[word]

vector_troquet = score('troquet')

print ( 'score troquet', vector_troquet, len(vector_troquet))


score troquet [-0.28589    -0.26518     0.076356   -0.017638    0.20075    -0.037748
 -0.20504     0.22178    -0.16894     0.4035     -0.34198     0.037411
 -0.081518   -0.1229     -0.51538     0.0088103   0.084543   -0.082374
 -0.38133    -0.07603     0.10483    -0.019706   -0.04029     0.37365
 -0.37959    -0.54548    -0.31914    -0.31262    -0.060444   -0.053712
  0.43251    -0.1079      0.23003    -0.066676   -0.29099    -0.081674
 -0.20116    -0.32752     0.091499    0.081911    0.051361    0.076685
 -0.071495   -0.070764    0.28938    -0.12803     0.066035    0.097801
 -0.016806    0.015525    0.084922   -0.23528    -0.017823    0.4899
 -0.15586    -0.14952    -0.30836     0.12642     0.10689     0.10892
 -0.17307     0.36345     0.10455     0.60661    -0.33281    -0.18994
  0.20595     0.074113    0.1325     -0.30423     0.076651    0.18055
 -0.29682    -0.17862    -0.43857    -0.002312    0.47742     0.096643
 -0.32785    -0.067572    0.013655   -0.20559     0.49405    -0.18791

In [14]:
#ai préféré utiliser le tokenizer de keras plutôt que celui de nltk car les valeurs filtrées par défaut comprennent l'apostrophe
#def tokenize(sentence):
#	tokens = text_to_word_sequence(sentence)
#	return tokens
	
#dataset['tokens']= dataset['text'].map(tokenize)
#dataset['tokens']

MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 5
texts = dataset['text']
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
#print('Shape of label tensor:', labels.shape)
dataset['tokens']= list(data)
dataset

Found 89 unique tokens.
Shape of data tensor: (73, 5)


,text,class,tokens,XEncoded,Ycodes
0,à prix modique,éco,"[0, 0, 20, 13, 21]",<map object at 0x000001F6055F4BE0>,"[0.0, 0.0, 1.0]"
1,aucun problème d'argent,cher,"[0, 0, 35, 36, 37]",<map object at 0x000001F6055F4940>,"[1.0, 0.0, 0.0]"
2,avantageux,éco,"[0, 0, 0, 0, 38]",<map object at 0x000001F6055F4CC0>,"[0.0, 0.0, 1.0]"
3,bas de gamme,éco,"[0, 0, 22, 1, 8]",<map object at 0x000001F6055F4AC8>,"[0.0, 0.0, 1.0]"
4,bistrot pas cher,éco,"[0, 0, 39, 3, 2]",<map object at 0x000001F6055F4F98>,"[0.0, 0.0, 1.0]"
5,bon,cher,"[0, 0, 0, 0, 5]",<map object at 0x000001F61A7A05F8>,"[1.0, 0.0, 0.0]"
6,bon marché,éco,"[0, 0, 0, 5, 14]",<map object at 0x000001F61A7A02E8>,"[0.0, 0.0, 1.0]"
7,bon rapport qualité prix,moyen,"[0, 5, 40, 15, 13]",<map object at 0x000001F61A7A07B8>,"[0.0, 1.0, 0.0]"
8,cher,cher,"[0, 0, 0, 0, 2]",<map object at 0x000001F61A7A06D8>,"[1.0, 0.0, 0.0]"
9,éco,éco,"[0, 0, 0, 0, 23]",<map object at 0x000001F61A7A0828>,"[0.0, 0.0, 1.0]"


# Préparer la couche d'intégration ( the Embedding layer)

Il s'agit de construire une matrice (nombre de mots x vecteur de 300) qui permette de retrouver le vecteur de chacun des mots du vocabulaire et dinitialiser la couche d'intégration



In [21]:
# prepare embedding matrix
EMBEDDING_DIM = 300
num_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    #embedding_vector = embeddings_index.get(word)
    embedding_vector = fr_model[word]    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)


KeyError: "word 'm’en' not in vocabulary"

Le prochain pavé permet de transformer les catégories "éco","moyen" et cher en vecteur.
Ces vecteurs sont rangés dans la colonne 'Ycodes' du dataset

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dataset['Ycodes'] = pd.Series(list(dummy_y))
dataset


,text,class,tokens,XEncoded,Ycodes
0,à prix modique,éco,"[20, 13, 21]",<map object at 0x000001F6055F4BE0>,"[0.0, 0.0, 1.0]"
1,aucun problème d'argent,cher,"[35, 36, 37]",<map object at 0x000001F6055F4940>,"[1.0, 0.0, 0.0]"
2,avantageux,éco,[38],<map object at 0x000001F6055F4CC0>,"[0.0, 0.0, 1.0]"
3,bas de gamme,éco,"[22, 1, 8]",<map object at 0x000001F6055F4AC8>,"[0.0, 0.0, 1.0]"
4,bistrot pas cher,éco,"[39, 3, 2]",<map object at 0x000001F6055F4F98>,"[0.0, 0.0, 1.0]"
5,bon,cher,[5],<map object at 0x000001F61A7A05F8>,"[1.0, 0.0, 0.0]"
6,bon marché,éco,"[5, 14]",<map object at 0x000001F61A7A02E8>,"[0.0, 0.0, 1.0]"
7,bon rapport qualité prix,moyen,"[5, 40, 15, 13]",<map object at 0x000001F61A7A07B8>,"[0.0, 1.0, 0.0]"
8,cher,cher,[2],<map object at 0x000001F61A7A06D8>,"[1.0, 0.0, 0.0]"
9,éco,éco,[23],<map object at 0x000001F61A7A0828>,"[0.0, 0.0, 1.0]"


###MODEL DEFINITION
Le réseau comprend trois couches.

Chaque phrase est représentée à l'aide d'un vecteur de longueur cinq à l'aide d'un vocabulaire à neuf mots.

In [8]:
########################### MODEL DEFINITION
input_dim = 10 #size of vocabular
output_dim = 1 #embedding_vecor_length
input_length = 5 #matrice width

def sequential_model():
    model = Sequential()
    model.add(Embedding(input_dim, output_dim, input_length=input_length))
    model.add(Convolution1D(64, 3, border_mode='same'))
    model.add(Convolution1D(32, 3, border_mode='same'))
    model.add(Convolution1D(16, 3, border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(180,activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = sequential_model()
model

d:\code\ml\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  if __name__ == '__main__':
d:\code\ml\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
  # Remove the CWD from sys.path while we load stuff.
d:\code\ml\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(16, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()


In [9]:
###### TRAINING + EVALUATION
x_train = XEncoded
y_train = dummy_y
batch_size = 5
model.fit(x_train, y_train,
          epochs=100,
          batch_size=batch_size)
x_test = x_train
y_test = y_train
loss,metric = model.evaluate(x_test, y_test, batch_size=batch_size)
print ("Loss = ", loss," Accuracy = ",metric)

NameError: name 'XEncoded' is not defined

In [ ]:
dataset['predicted'] = list(model.predict(XEncoded)) 

def decodeY(L):
	if L[0]>L[1] and L[0]>L[2]: 
		return "cher"
	elif L[2]>L[1] and L[2]>L[0]:
		return "éco"
	else:
		return "moyen"
	
dataset['resultat']=list(map(decodeY,dataset['predicted']))
dataset
